In [209]:
import numpy as np
import glob
import os

In [210]:
# Annotate in the style of ILSVRC
# <annotation>
# 	<folder>ILSVRC2017_VID_train_0000/ILSVRC2017_train_00000000</folder>
# 	<filename>000000</filename>
# 	<source>
# 		<database>ILSVRC_2017</database>
# 	</source>
# 	<size>
# 		<width>1280</width>
# 		<height>720</height>
# 	</size>
# 	<object>
# 		<trackid>0</trackid>
# 		<name>n01503061</name>
# 		<bndbox>
# 			<xmax>892</xmax>
# 			<xmin>675</xmin>
# 			<ymax>506</ymax>
# 			<ymin>296</ymin>
# 		</bndbox>
# 		<occluded>0</occluded>
# 		<generated>0</generated>
# 	</object>
# </annotation>
import xml.etree.ElementTree as gfg
from xml.dom import minidom

def create_imagenet_style_xml(folder, 
                              filename, 
                              source, 
                              width, height, 
                              trackid, 
                              name, xmax, xmin, ymax, ymin):
    root = gfg.Element('annotation')

    folder_elem = gfg.Element("folder")
    folder_elem.text = folder
    root.append(folder_elem)

    filename_elem = gfg.Element("filename")
    filename_elem.text = filename
    root.append(filename_elem)

    source_elem = gfg.Element("source")
    source_elem.text = source
    root.append(source_elem)

    size_elem = gfg.Element("size")
    width_elem = gfg.Element("width")
    width_elem.text = width
    size_elem.append(width_elem)

    height_elem = gfg.Element("height")
    height_elem.text = height
    size_elem.append(height_elem)
    root.append(size_elem)

    objekt_elem = gfg.Element("object")
    trackid_elem = gfg.Element("trackid")
    trackid_elem.text = trackid
    objekt_elem.append(trackid_elem)

    name_elem = gfg.Element("name")
    name_elem.text= name
    objekt_elem.append(name_elem)

    bndbox_elem = gfg.Element("bndbox")
    xmax_elem = gfg.Element("xmax")
    xmax_elem.text = xmax
    bndbox_elem.append(xmax_elem)

    xmin_elem = gfg.Element("xmin")
    xmin_elem.text = xmin
    bndbox_elem.append(xmin_elem)

    ymax_elem = gfg.Element("ymax")
    ymax_elem.text = ymax
    bndbox_elem.append(ymax_elem)

    ymin_elem = gfg.Element("ymin")
    ymin_elem.text = ymin
    bndbox_elem.append(ymin_elem)
    
    objekt_elem.append(bndbox_elem)

    occluded_elem = gfg.Element("occluded")
    occluded_elem.text = "0"
    objekt_elem.append(occluded_elem)

    generated_elem = gfg.Element("generated")
    generated_elem.text = "0"
    objekt_elem.append(generated_elem)
    
    root.append(objekt_elem)

    xml_tree = gfg.tostring(root)
    reparsed = minidom.parseString(xml_tree)
    #print(reparsed.childNodes[0].toprettyxml())
    output_xml = reparsed.childNodes[0]
    return output_xml

In [211]:
# Imagenet organisation:
# For each video theres a train folder with annotations for each frame
# preprocess imagenet vid:
# takes all xmls for the given vid
# selects two at random
# fo rthese two it extracts the anntoation from the xml
# then for each it extracts the object annotation from which it extracts the object.

# then for each it extracts the bounding box
# it does nothing to the data or the folder, since those are externally provided.
# AND the filename matches the name of the annotation file, e.g. 000000.xml
# And that is it.

# So what we do:
# For every sequence we create an annotation folder. This folder contains xml annotations,
# where the name corresponds to the file name of the frame that it describes

In [212]:
# Create an ImageSet for training: train.txt
# Create an ImageSet for testing: test.txt

In [213]:
# Train on G2 Eval sequences
# Test on G1 Eval Sequences

In [214]:
#G1
g1_path = "/home/oole/git/projects-to-consume/chokepoint-bbs/G1"
g1_all_sequence_files = glob.glob(os.path.join(g1_path, "*.txt"))
g1_all_sequence_files.sort()
g1_train_sequence_file = [file for file in g1_all_sequence_files if "train" in file][0]
g1_eval_sequences_files = [file for file in g1_all_sequence_files if "eval" in file]
if len(g1_train_sequence_file) is None or len(g1_eval_sequences_files) < 10:
    print("G2 sequences?")

g1_eval_dict = {}
for eval_sequence in g1_eval_sequences_files:
    eval_scenes, eval_frame_names, eval_xmins, eval_ymins, eval_xmaxs, eval_ymaxs = [], [], [] ,[] ,[], []
    with open(eval_sequence) as file:
        for line in file:
            line = line.strip('\n')
            file_path, ymin, xmin, width, height  = line.split(',')
            _, scene, frame_name = file_path.split("/")
            eval_scenes.append(scene)
            eval_frame_names.append(frame_name)
            eval_xmins.append(int(xmin))
            eval_ymins.append(int(ymin))
            eval_xmaxs.append(int(xmin)+int(width))
            eval_ymaxs.append(int(ymin)+int(height))
    if not len(eval_scenes) == len(eval_frame_names) == len(eval_xmins) == len(eval_ymins) == len(eval_xmaxs) == len(eval_ymaxs):
        print('probem')
        break
    g1_eval_dict[eval_sequence] = {
        'scenes': eval_scenes, 
        'frame_names': eval_frame_names,
        'xmins': eval_xmins,
        'ymins': eval_ymins,
        'xmaxs': eval_xmaxs,
        'ymaxs': eval_ymaxs}

In [215]:
#G2
g2_path = "/home/oole/git/projects-to-consume/chokepoint-bbs/G2"
g2_all_sequence_files = glob.glob(os.path.join(g2_path, "*.txt"))
g2_all_sequence_files.sort()
g2_train_sequence_file = [file for file in g2_all_sequence_files if "train" in file][0]
g2_eval_sequences_files = [file for file in g2_all_sequence_files if "eval" in file]
if len(g2_train_sequence_file) is None or len(g2_eval_sequences_files) < 10:
    print("G2 sequences?")
    
g2_eval_dict = {}
for eval_sequence in g2_eval_sequences_files:
    eval_scenes, eval_frame_names, eval_xmins, eval_ymins, eval_xmaxs, eval_ymaxs = [], [], [] ,[] ,[], []
    with open(eval_sequence) as file:
        for line in file:
            line = line.strip('\n')
            file_path, ymin, xmin, width, height  = line.split(',')
            _, scene, frame_name = file_path.split("/")
            eval_scenes.append(scene)
            eval_frame_names.append(frame_name)
            eval_xmins.append(int(xmin))
            eval_ymins.append(int(ymin))
            eval_xmaxs.append(int(xmin)+int(width))
            eval_ymaxs.append(int(ymin)+int(height))
    if not len(eval_scenes) == len(eval_frame_names) == len(eval_xmins) == len(eval_ymins) == len(eval_xmaxs) == len(eval_ymaxs):
        print('probem')
        break
    g2_eval_dict[eval_sequence] = {
        'scenes': eval_scenes, 
        'frame_names': eval_frame_names,
        'xmins': eval_xmins,
        'ymins': eval_ymins,
        'xmaxs': eval_xmaxs,
        'ymaxs': eval_ymaxs}

In [216]:
len(g2_eval_sequences_files)


216

In [217]:
def expand_bbox(xmin,ymin,xmax, ymax):
    expand_by =10
    xmin = xmin-expand_by
    xmax = xmax+expand_by
    ymin = ymin-expand_by
    ymax = ymax+expand_by
    return xmin, ymin, xmax, ymax

In [218]:
base_path = "/home/oole/Data/Chokepoint"

annotation_path = os.path.join(base_path,"annotation")

relative_path_to_sequences = "data/"


def write_eval_dict_to_xml(eval_dict, group, mode, key_list=None):
    count_scene_dict = {}
#     found_one_with_point = False
#     printed_one_without_point = False
    if key_list is None:
        dict_keys =eval_dict.keys()
    else:
        dict_keys = key_list
        print("using supplied key list")
    for key in dict_keys:
        sequence = eval_dict[key]
        unique_scenes = np.unique(sequence['scenes'])
        assert len(unique_scenes) == 1
        sub_scene_dir = unique_scenes[0]
        scene_split = sub_scene_dir.split('_')
        if '.' in sub_scene_dir:
#             print(sub_scene_dir)
            three_dir_layers = True
#             found_one_with_point = True
            in_between_scene_dir = scene_split[0] + '_' + scene_split[1] + "_" + scene_split[2].split('.')[0]
            top_scene_dir = scene_split[0] + '_' + scene_split[1]
#             print("top: {}".format(top_scene_dir))
#             print("in_between_scene_dir: {}".format(in_between_scene_dir))
#             print("sub: {}".format(sub_scene_dir))
        else:
            top_scene_dir = scene_split[0] + '_' + scene_split[1]
            three_dir_layers = False
#             if not printed_one_without_point:
#                 print("normal top: {}".format(top_scene_dir))
#                 print("sub_scene_dir: {}".format(sub_scene_dir))
#                 printed_one_without_point = True
            
#         print(top_scene_dir)
        frames = sequence['frame_names']
        xmins = sequence['xmins']
        ymins = sequence['ymins']
        xmaxs = sequence['xmaxs']
        ymaxs = sequence['ymaxs']
        key_split = key.split('_')
        sequence_name = key_split[2] + '_' + key_split[3].split('.')[0]
        sequence_dir = sub_scene_dir + "_" + sequence_name
        if three_dir_layers:
            sequence_dir_path = os.path.join(annotation_path, group, mode, 
                                             top_scene_dir, in_between_scene_dir,sequence_dir)
        else:
            sequence_dir_path = os.path.join(annotation_path, group, mode, top_scene_dir, sequence_dir)
        if not os.path.exists(sequence_dir_path):
            os.makedirs(sequence_dir_path)
#         print(sequence_dir_path)
        for idx, frame in enumerate(frames):
#             print(frame)
            frame_number = frame.split('.')[0]
#             print(frame_number)
            xmin = xmins[idx]
            ymin = ymins[idx]
            xmax = xmaxs[idx]
            ymax = ymaxs[idx]
            xmin,ymin,xmax, ymax =  expand_bbox(xmin,ymin,xmax, ymax)
            if three_dir_layers:
                folder = os.path.join(relative_path_to_sequences, top_scene_dir, in_between_scene_dir, sub_scene_dir)
            else:
                folder = os.path.join(relative_path_to_sequences, top_scene_dir, sub_scene_dir)
            filename = frame_number
            source = "Chokepoint_Dataset"
            width = "800"
            height = "600"
            trackid = "0"
            name = "face"
            xml = create_imagenet_style_xml(folder,
                                      filename, 
                                      source, 
                                      width, 
                                      height, 
                                      trackid, 
                                      name, 
                                      str(xmax), 
                                      str(xmin), 
                                      str(ymax), 
                                      str(ymin))
            xml_file_path = os.path.join(sequence_dir_path, frame_number + ".xml")
#             break
            with open(xml_file_path, "w") as file:
                file.write(xml.toprettyxml())
            
            
#             print(xml_file_path)
#             print(xml.toprettyxml())



In [219]:
# write_eval_dict_to_xml(g2_eval_dict, group = "G1", mode = "train")
# 	<folder>data/P2L_S2_C2.2/P2L_S2_C2.2</folder>

In [220]:
g2_keys = g2_eval_dict.keys()
# print(len(keys))
# seq_numbers = []
# for key in keys:
#     seq_number = key.split("/")[-1].split("_")[-1].split(".")[0]
#     print(seq_number)
# per scene there are 25 sequences
# There are 
g2_num_val =  int(len(g2_keys) * 0.2)
g2_num_val
import random
random.seed(1)
g2_keys_list = list(g2_keys)
random.shuffle(g2_keys_list)
# for key in keys_list:
#     print(key)
g2_val_keys = g2_keys_list[:g2_num_val]
g2_train_keys = g2_keys_list[g2_num_val:]
len(g2_val_keys)
for key in g2_val_keys:
    if key in g2_train_keys:
        print("alarm")
for key in g2_train_keys:
    if key in g2_val_keys:
        print("alarm")


In [223]:
# for key in g1_eval_dict.keys():
#     elem = g1_eval_dict[key]
#     print(elem)
#     break

In [224]:
# G1:
# use sequence from g2_eval dict (which is the train partition) for validation
# 
write_eval_dict_to_xml(g2_eval_dict, group = "G1", mode = "train", key_list=g2_train_keys)
write_eval_dict_to_xml(g2_eval_dict, group = "G1", mode = "val", key_list=g2_val_keys)
write_eval_dict_to_xml(g1_eval_dict, group = "G1", mode = "test", key_list=g1_eval_dict.keys())

using supplied key list
using supplied key list
using supplied key list


In [225]:
g1_keys = g1_eval_dict.keys()
# print(len(keys))
# seq_numbers = []
# for key in keys:
#     seq_number = key.split("/")[-1].split("_")[-1].split(".")[0]
#     print(seq_number)
# per scene there are 25 sequences
# There are 
g1_num_val =  int(len(g1_keys) * 0.2)
g1_num_val
import random
random.seed(1)
g1_keys_list = list(g1_keys)
random.shuffle(g1_keys_list)
# for key in keys_list:
#     print(key)
g1_val_keys = g1_keys_list[:g1_num_val]
g1_train_keys = g1_keys_list[g1_num_val:]
len(g1_val_keys)
for key in g1_val_keys:
    if key in g1_train_keys:
        print("alarm")
for key in g1_train_keys:
    if key in g1_val_keys:
        print("alarm")


In [226]:
# G2:
write_eval_dict_to_xml(g1_eval_dict, group = "G2", mode = "train", key_list=g1_train_keys)
write_eval_dict_to_xml(g1_eval_dict, group = "G2", mode = "val", key_list=g1_val_keys)
write_eval_dict_to_xml(g2_eval_dict, group = "G2", mode = "test", key_list=g2_eval_dict.keys())

using supplied key list
using supplied key list
using supplied key list


In [65]:
# load_roidb_chokepoint(subset)...
CHOKEPOINT_ROOT = "/home/oole/Data/Chokepoint"
subset = "train"


In [128]:
import glob
paths = sorted(glob.glob(subset_path + "*/*/*/*"))
vid_names_1 = [path for path in paths if "xml" not in path and "seq" in path]
for vid_name in vid_names_1:
    if "P2L_S1_C1.1_seq_159" in vid_name:
        print(vid_name)
vid_names_1 = ["/".join(v.split("/")[-3:]) for v in vid_names_1]
                
paths = sorted(glob.glob(subset_path + "*/*/*"))
vid_names_2 = [path for path in paths if "xml" not in path and "seq" in path]
for vid_name in vid_names_2:
    if "P2L_S1_C1.1_seq_159" in vid_name:
        print(vid_name)
vid_names_2 = ["/".join(v.split("/")[-2:]) for v in vid_names_2]

/home/oole/Data/Chokepoint/annotation/G1/train/P2L_S1/P2L_S1_C1/P2L_S1_C1.1_seq_159


In [131]:
vid_names = list(vid_names_2 + vid_names_1)
vid_names

['P1E_S1/P1E_S1_C1_seq_1',
 'P1E_S1/P1E_S1_C1_seq_10',
 'P1E_S1/P1E_S1_C1_seq_11',
 'P1E_S1/P1E_S1_C1_seq_12',
 'P1E_S1/P1E_S1_C1_seq_13',
 'P1E_S1/P1E_S1_C1_seq_14',
 'P1E_S1/P1E_S1_C1_seq_15',
 'P1E_S1/P1E_S1_C1_seq_16',
 'P1E_S1/P1E_S1_C1_seq_17',
 'P1E_S1/P1E_S1_C1_seq_18',
 'P1E_S1/P1E_S1_C1_seq_19',
 'P1E_S1/P1E_S1_C1_seq_2',
 'P1E_S1/P1E_S1_C1_seq_20',
 'P1E_S1/P1E_S1_C1_seq_21',
 'P1E_S1/P1E_S1_C1_seq_22',
 'P1E_S1/P1E_S1_C1_seq_23',
 'P1E_S1/P1E_S1_C1_seq_24',
 'P1E_S1/P1E_S1_C1_seq_25',
 'P1E_S1/P1E_S1_C1_seq_3',
 'P1E_S1/P1E_S1_C1_seq_4',
 'P1E_S1/P1E_S1_C1_seq_5',
 'P1E_S1/P1E_S1_C1_seq_6',
 'P1E_S1/P1E_S1_C1_seq_7',
 'P1E_S1/P1E_S1_C1_seq_8',
 'P1E_S1/P1E_S1_C1_seq_9',
 'P1E_S2/P1E_S2_C2_seq_26',
 'P1E_S2/P1E_S2_C2_seq_27',
 'P1E_S2/P1E_S2_C2_seq_28',
 'P1E_S2/P1E_S2_C2_seq_29',
 'P1E_S2/P1E_S2_C2_seq_30',
 'P1E_S2/P1E_S2_C2_seq_31',
 'P1E_S2/P1E_S2_C2_seq_32',
 'P1E_S2/P1E_S2_C2_seq_33',
 'P1E_S2/P1E_S2_C2_seq_34',
 'P1E_S2/P1E_S2_C2_seq_35',
 'P1E_S2/P1E_S2_C2_seq_36',
 